In [29]:
import re
import pandas as pd
import numpy as np
import MySQLdb
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold, KFold
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Dropout, LSTM, Merge, Input, Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.utils import np_utils
from sklearn.externals import joblib
from sqlalchemy import create_engine

In [30]:
def binary(Y):
    Y[np.where(Y > 0)] = 1
    Y[np.where(Y <= 0)] = 0
    Y = Y.astype('int64')
    f = np.bincount(Y)
    print(f/np.sum(f))
    return Y

In [31]:
def get_shift(data, shift_offset=2):
    g = data.as_matrix(columns=['gbp_gradient']).reshape(-1)
    for i in range(1, shift_offset+1):
        data['gbp_gradient_p_'+str(i)] = shift(g, i, cval=3)

In [32]:
data = pd.read_csv('newstitle_gbp(uk-news,world,business,technology,money_2000-2016).csv')
data['gbp_gradient'] = np.gradient(data.as_matrix(columns=['gbp']).reshape(-1))
get_shift(data, shift_offset=2)

In [33]:
data.head()

,timestamp,title,gbp,gbp_gradient,gbp_gradient_p_1,gbp_gradient_p_2
0,946656000,"Big Macs, small horizons Y2K force outstrips s...",1.615700,-0.000700,3.000000,3.000000
1,946742400,Cock-fighting? Yes please Samaritans split ove...,1.615000,0.005512,-0.000700,3.000000
2,946828800,Hunters threaten Jospin with new battle of the...,1.626724,0.011001,0.005512,-0.000700
3,946915200,Leader: German sleaze inquiry Burundi peace in...,1.637002,0.005804,0.011001,0.005512
4,947001600,Women in record South Pole walk Deaths no coin...,1.638331,0.005379,0.005804,0.011001


In [34]:
Y = data.as_matrix(columns=['gbp_gradient']).reshape(-1)
#Y = shift(Y, -3, cval=3)
Y = binary(Y)
num_classes = 2

[ 0.480071  0.519929]


In [35]:
X_p = data.as_matrix(columns=['gbp_gradient_p_1', 'gbp_gradient_p_2'])
X_p[:,0] = binary(X_p[:,0])
X_p[:,1] = binary(X_p[:,1])
# X_p = data.as_matrix(columns=['gbp_gradient','gbp_gradient_p_1'])
# X_p[:,0] = binary(X_p[:,0])
# X_p[:,1] = binary(X_p[:,1])

[ 0.4795869  0.5204131]
[ 0.4795869  0.5204131]


In [36]:
feature_name = 'title'
vectorizer = TfidfVectorizer(min_df=50, ngram_range=(1, 4))
X = vectorizer.fit_transform(data[feature_name].tolist())
X

<6197x2730 sparse matrix of type '<class 'numpy.float64'>'
	with 476823 stored elements in Compressed Sparse Row format>

In [37]:
with_news = True

In [38]:
if with_news:
    X = hstack([X, X.power(2), X.power(3), X.power(4), X_p])
else:
    X = X_p

In [39]:
def create_model():
    model = Sequential()
    model.add(Dense(10, activation='tanh', input_shape=(X.shape[1],), bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='tanh', bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='tanh', bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='tanh', bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='RMSprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()
    return model

In [40]:
# output = pd.DataFrame()
# output['timestamp'] = data['timestamp']
# output['gbp'] = data['gbp']
# output['gbp_gradient'] = data['gbp_gradient']
# output['gbp_gradient_binary'] = Y

In [41]:
def show_result(X_test, y_test, binary_pred, test_idx):
    target_names = ['decline','up']
    pred = np.argmax(model.predict(X_test), axis=1)
    binary_pred[test_idx] = pred
    #print(confusion_matrix(y_test, pred, labels=[0,1]))
    mat = confusion_matrix(y_test, pred, labels=[0,1])
    report = classification_report(y_test, pred, target_names=target_names) + '\n'
    return report

In [53]:
binary_pred = np.zeros(6197) #record pred value
skf = StratifiedKFold(n_splits=5, shuffle=True)
acc = []
num_epochs = 37 
history = ''
for i in range(1):
    for train_idx, test_idx in skf.split(X, Y):
        print ("Running Fold")
        model = create_model()
        if with_news:
            x = X.toarray()
        else:
            x = X
        y = np_utils.to_categorical(Y, num_classes)
        report = model.fit(x[train_idx], y[train_idx], 
                        epochs=num_epochs,
                        validation_data =(x[test_idx], y[test_idx]),
                        batch_size=7000,
                        verbose = 1)
        acc.append(model.evaluate(x[test_idx], y[test_idx], batch_size=2000)[1])
        history +=show_result(x[test_idx], Y[test_idx], binary_pred, test_idx)
    print(i)
print('acc:%.4f'%(np.mean(acc)))
print(history)

Running Fold
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_216 (Dense)            (None, 10)                109230    
_________________________________________________________________
dropout_173 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_217 (Dense)            (None, 10)                110       
_________________________________________________________________
dropout_174 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_218 (Dense)            (None, 10)                110       
_________________________________________________________________
dropout_175 (Dropout)        (None, 10)                0         
_________________________________________________________________
dense_219 (Dense)            (None, 10)                110     

4958/4958 [==============================] - 0s - loss: 0.6645 - acc: 0.5889 - val_loss: 0.6794 - val_acc: 0.5948
Epoch 19/37
4958/4958 [==============================] - 0s - loss: 0.6640 - acc: 0.5954 - val_loss: 0.6789 - val_acc: 0.6029
Epoch 20/37
4958/4958 [==============================] - 0s - loss: 0.6649 - acc: 0.5922 - val_loss: 0.6780 - val_acc: 0.5948
Epoch 21/37
4958/4958 [==============================] - 0s - loss: 0.6590 - acc: 0.6129 - val_loss: 0.6775 - val_acc: 0.6094
Epoch 22/37
4958/4958 [==============================] - 0s - loss: 0.6573 - acc: 0.6158 - val_loss: 0.6769 - val_acc: 0.6118
Epoch 23/37
4958/4958 [==============================] - 0s - loss: 0.6555 - acc: 0.6172 - val_loss: 0.6760 - val_acc: 0.6069
Epoch 24/37
4958/4958 [==============================] - 0s - loss: 0.6503 - acc: 0.6196 - val_loss: 0.6751 - val_acc: 0.6021
Epoch 25/37
4958/4958 [==============================] - 0s - loss: 0.6525 - acc: 0.6216 - val_loss: 0.6751 - val_acc: 0.6077
Epoc

In [44]:
model = create_model()
x = X.toarray()
y = np_utils.to_categorical(Y, num_classes)
model.fit(x, y, 
        epochs=num_epochs,
        batch_size=7000,
        verbose = 1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 10)                109230    
_________________________________________________________________
dropout_29 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_37 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_30 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_31 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 10)                110       
__________

In [45]:
# model.save('model.h5')
# joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

In [43]:
#output['gbp_gradient_binary_pred'] = binary_pred
#output.to_pickle('output.pkl')